In [2]:
import json
import math
import os
import random
import re
import shutil
import sys
import time
from collections import defaultdict
from pathlib import Path
from datetime import datetime

from loguru import logger


data_dir = Path('/data/comm/RUC-NLPIR/FlashRAG_datasets/nq')
train_file = data_dir / 'train.parquet'
test_file = data_dir / 'test.parquet'

def load_data(file_path):
    if file_path.suffix == '.jsonl':
        with open(file_path, 'r', encoding='utf-8') as f:
            data = [json.loads(line) for line in f]
    elif file_path.suffix == '.parquet':
        import pyarrow.parquet as pq
        table = pq.read_table(file_path)
        data = table.to_pandas().to_dict(orient='records')
    else:
        raise ValueError(f"Unsupported file format: {file_path.suffix}")
    return data

data = load_data(train_file)
print(data[0])
print(data[-1])


{'id': 'train_0', 'question': 'total number of death row inmates in the us?', 'golden_answers': array(['2,718'], dtype=object), 'data_source': 'nq', 'prompt': array([{'content': 'Answer the given question. You must conduct reasoning inside <think> and </think> first every time you get new information. After reasoning, if you find you lack some knowledge, you can call a search engine by <search> query </search> and it will return the top searched results between <information> and </information>. You can search as many times as your want. If you find no further external knowledge needed, you can directly provide the answer inside <answer> and </answer>, without detailed illustrations. For example, <answer> Beijing </answer>. Question: total number of death row inmates in the us?\n', 'role': 'user'}],
      dtype=object), 'ability': 'fact-reasoning', 'reward_model': {'ground_truth': {'target': array(['2,718'], dtype=object)}, 'style': 'rule'}, 'extra_info': {'index': 0, 'split': 'train'}}

In [3]:
data_dir = Path('/data/comm/RUC-NLPIR/FlashRAG_datasets/nq_hotpotqa_train')
train_file = data_dir / 'train.parquet'
test_file = data_dir / 'test.parquet'
data = load_data(train_file)
print(f'{type(data[0]) = }')
print(data[0])
print()
print(data[-1])

type(data[0]) = <class 'dict'>
{'id': 'train_0', 'question': 'total number of death row inmates in the us?', 'golden_answers': array(['2,718'], dtype=object), 'data_source': 'nq', 'prompt': array([{'content': 'Answer the given question. You must conduct reasoning inside <think> and </think> first every time you get new information. After reasoning, if you find you lack some knowledge, you can call a search engine by <search> query </search> and it will return the top searched results between <information> and </information>. You can search as many times as your want. If you find no further external knowledge needed, you can directly provide the answer inside <answer> and </answer>, without detailed illustrations. For example, <answer> Beijing </answer>. Question: total number of death row inmates in the us?\n', 'role': 'user'}],
      dtype=object), 'ability': 'fact-reasoning', 'reward_model': {'ground_truth': {'target': array(['2,718'], dtype=object)}, 'style': 'rule'}, 'extra_info': 

In [13]:
print(len(data[-1]['metadata']['supporting_facts']))
print(len(data[-1]['metadata']['context']['sentences']))
print(data[-1]['metadata']['context']['sentences'][0])
print(data[-1]['metadata']['context']['sentences'][-1])

2
10
['The Vietnam national cricket team represents Vietnam in international cricket.'
 ' It will debut in the cricket tournament at the 2017 Southeast Asian Games in Kuala Lumpur, Malaysia.']
['The Mozambique national cricket team is the team that represents the country of Mozambique in international cricket matches.'
 ' The Mozambique national cricket team, which is administered by the Mozambican Cricket Association, is an affiliate member of the International Cricket Council (ICC).'
 ' Mozambique is also a member of the African Cricket Association.'
 ' The Mozambique national cricket team has competed in the World Cricket League Africa Region and the ICC Africa Twenty20 Championship.']
